In [1]:
import numpy as np
import pandas as pd

import os

import matplotlib.pyplot as plt

import cv2 as cv

from sklearn.model_selection import train_test_split 

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
#1. 데이터 준비_전처리(data set processing)
#1-1. 일별 초당 평균발전량 (date and time&AVG power)데이터 가공
#1-1-1. csv파일 읽기
power_10_df = pd.read_csv("./1Low-res_sky_images_kaggle/Power_measurements/2019_09_10.csv")
power_11_df = pd.read_csv("./1Low-res_sky_images_kaggle/Power_measurements/2019_09_11.csv")
power_12_df = pd.read_csv("./1Low-res_sky_images_kaggle/Power_measurements/2019_09_12.csv")

#1-1-2. 날짜&시간을 알려주는 컬럼['date and time']의 자료형태를 시간으로 바꾸기
power_10_df['Date and Time'] = pd.to_datetime(power_10_df['Date and Time'])
power_11_df['Date and Time'] = pd.to_datetime(power_11_df['Date and Time'])
power_12_df['Date and Time'] = pd.to_datetime(power_12_df['Date and Time'])

#1-1-3. power_10_df, power_11_df, power_12_df을 합치기
power_df = pd.concat([power_10_df, power_11_df, power_12_df], axis = 0)

#1-1-4.결측값 제거하기
power_df.dropna(inplace=True)

#1-1-5. 인덱스'Date and Time'에서 중복되는 경우에 대해서 groupby로 묶어서 평균값 주기 
power_df_avg = power_df.groupby('Date and Time').mean()
      
#1-1-6. 인덱스'Date and Time'를 컬럼'DateTime'으로 만들기
power_df_avg['DateTime'] = power_df_avg.index

#1-1-7. 인덱스'Date and Time'를 reset하기
power_df_avg = power_df_avg.reset_index(drop=True)     

C:\Users\user\AppData\Local\Temp\ipykernel_15468\197603555.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  power_10_df['Date and Time'] = pd.to_datetime(power_10_df['Date and Time'])
C:\Users\user\AppData\Local\Temp\ipykernel_15468\197603555.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  power_11_df['Date and Time'] = pd.to_datetime(power_11_df['Date and Time'])
C:\Users\user\AppData\Local\Temp\ipykernel_15468\197603555.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  power_12_df['Date and Time'] = pd.to_datetime(power_12_df['Date and Time'])


In [3]:
#1-2.Image파일의 파일명 번호에 따른 Image의 촬영 시분초(사진번호, 시, 분, 초 )데이터의 가공
#1-2-1. csv파일 읽기
time_10_df = pd.read_csv("./1Low-res_sky_images_kaggle/Sky_images/10_09_2019/time.csv")
time_11_df = pd.read_csv("./1Low-res_sky_images_kaggle/Sky_images/11_09_2019/time.csv")
time_12_df = pd.read_csv("./1Low-res_sky_images_kaggle/Sky_images/11_09_2019/time.csv")

#1-2-2. csv 파일상에서는 사진번호1~3000까지의 데이터가 있지만, 실제 사진은 1~2999까지 있으므로 사진번호3000 제거하기
time_10_df = time_10_df[:-1]
time_11_df = time_11_df[:-1]
time_12_df = time_12_df[:-1]

#1-2-3. 날짜, 시간을 알려주는 컬럼['date and time']의 생성
time_10_df['Date and Time'] = pd.to_datetime(time_10_df[['Hours', 'Minutes', 'Seconds']].assign(Day=10, Month=9, Year=2019))
time_10_df = time_10_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)
time_11_df['Date and Time'] = pd.to_datetime(time_11_df[['Hours', 'Minutes', 'Seconds']].assign(Day=11, Month=9, Year=2019))
time_11_df = time_11_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)
time_12_df['Date and Time'] = pd.to_datetime(time_12_df[['Hours', 'Minutes', 'Seconds']].assign(Day=12, Month=9, Year=2019))
time_12_df = time_12_df.drop(['Hours', 'Minutes', 'Seconds'], axis=1)

#1-2-4. 사진파일이 있는 폴더경로를 알려주는 컬럼['FilePath']의 생성
folder = './1Low-res_sky_images_kaggle/Sky_images/10_09_2019/Images/'
time_10_df['FilePath'] = [os.path.join(folder, str(image_number) + '.jpg').replace('\\', '/') for image_number in time_10_df['Image Number']]
folder = './1Low-res_sky_images_kaggle/Sky_images/11_09_2019/Images/'
time_11_df['FilePath'] = [os.path.join(folder, str(image_number) + '.jpg').replace('\\', '/') for image_number in time_11_df['Image Number']]
folder = './1Low-res_sky_images_kaggle/Sky_images/12_09_2019/Images/'
time_12_df['FilePath'] = [os.path.join(folder, str(image_number) + '.jpg').replace('\\', '/') for image_number in time_12_df['Image Number']]

#1-2-5. 결측값 제거하기
time_10_df.dropna(inplace=True)
time_11_df.dropna(inplace=True)
time_12_df.dropna(inplace=True)

#1-2-6. 분단위로 데이터 정리를 위해 컬럼[Date and Time]에 floor을 분에 적용하여 분 이하에 값(초)가 없어진 컬럼[DateTime]을 생성
time_10_df['DateTime'] = time_10_df['Date and Time'].dt.floor('min')
time_11_df['DateTime'] = time_11_df['Date and Time'].dt.floor('min')
time_12_df['DateTime'] = time_12_df['Date and Time'].dt.floor('min')

#1-2-7. 분단위로 데이터 정리를 위해 컬럼[DateTime]을 groupby로 같은 분단위 시간끼리 묶기
grouped_10 = time_10_df.groupby('DateTime')
grouped_11 = time_11_df.groupby('DateTime')
grouped_12 = time_12_df.groupby('DateTime')

#1-2-8. 같은 분끼리 묶인 grouped_10의 컬럼'FilePath'에 ','를 기준으로 join하기
aggregated_10 = grouped_10['FilePath'].agg(lambda x: ','.join(x))
aggregated_11 = grouped_11['FilePath'].agg(lambda x: ','.join(x))
aggregated_12 = grouped_12['FilePath'].agg(lambda x: ','.join(x))

#1-2-9. reset_index로 인덱스를 다시 0~부터 리셋하고, 컬럼'FilePath'의 컬럼명을 'FilePaths'로 변경하기
result_10 = aggregated_10.reset_index().rename(columns={'FilePath': 'FilePaths'})
result_11 = aggregated_11.reset_index().rename(columns={'FilePath': 'FilePaths'})
result_12 = aggregated_12.reset_index().rename(columns={'FilePath': 'FilePaths'})

#1-2-10. 컬럼'FilePaths'의 파일경로들이 몇개인지 ','를 기준으로 len()으로 수를 세어 컬럼'NumberFiles'를 생성 후, 값을 주기
result_10['NumberFiles'] = result_10['FilePaths'].str.split(',').str.len()
result_11['NumberFiles'] = result_11['FilePaths'].str.split(',').str.len()
result_12['NumberFiles'] = result_12['FilePaths'].str.split(',').str.len()

#1-2-11. result_10,result_11,result_12(10,11,12일의 사진에 대한 정보)를 CONCAT으로 이어붙이기
images_df = pd.concat([result_10, result_11, result_12], axis = 0)

In [4]:
#1-3. 일별 초당 평균발전량 (date and time&AVG power)데이터와 Image파일의 파일명 번호에 따른 Image의 촬영 시분초(사진번호, 시, 분, 초 )데이터의 가공
#1-3-1. power_df_avg와 images_df를 컬럼'DateTime'를 기준으로 MERGE하기
merged_df = pd.merge(power_df_avg, images_df, on='DateTime')

In [5]:
#1-4.1000W/m^2단위로 category를 나누기
#.1-4-1. 1000W/m^2단위별로 bins를 정하고, bins에 해당하는 labels를 정해주고, cut으로 해당 bins을 기준으로 labels를 해주는 컬럼'category'를 생성 후, 값 주기
bins = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000]     #labels의 갯수는 bins의 갯수보다 1개가 적어야함.
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']               #뒤에서 tensorflow.keras.utils.to_categorical을 쓰려면 labels는 int(숫자)여야함.  #한글 및 영어 혼용 불가
merged_df['category'] = pd.cut(merged_df['AVG power'], bins=bins, labels=labels)

In [6]:
#1-5. array형태의 독립변수(X) 준비
#1-5-1. 파일 경로가 담긴 리스트형 자료 준비
        #데이터프레임merged_df의 컬럼'FilePaths'를 뽑아 컬럼'NumberFiles'의 개수만큼 있는 파일경로들을 split(',')으로 분리하여 리스트file_list에 담기
file_list= []
for i in range(merged_df.shape[0]):
    for j in range( merged_df['NumberFiles'][i]):
        file=merged_df['FilePaths'][i].split(',')[j]
        file_list.append(file)
len(file_list)

8997

In [7]:
#1-5-2. 빈 X를 만든 후, 리스트file_list상의 파일경로에 있는 이미지파일을 matplotlib.pyplot의 imread로 읽어들이고, cv2를 통해 이미지를 사이즈를 (150,150)으로 조절한 후 값을 X에 바꿔넣기
X=np.zeros((8997,150,150,3))              #비어있는 어레이X만든 후, 교체해 넣기

for idx,file in enumerate(file_list):
    dt=plt.imread(file)
    dt=cv.resize(dt,(150,150)).reshape(1,150,150,-1)
    X[idx] = dt[:,:,:,:3]

In [8]:
#1-6. list형태의 종속변수(y) 준비
#1-6-1.컬럼'category'를 컬럼'NumberFiles'의 개수만큼 있는 반복하도록 값을 뽑아 리스트y에 담기
y=[]
for i in range(merged_df.shape[0]):
    for j in range( merged_df['NumberFiles'][i]):
        y_=merged_df['category'][i]
        y.append(y_)  

In [9]:
#2. 머신러닝(딥러닝) 하기
#2-1. train데이터, validation데이터, test데이터 세트 만들기
#2-1-1. sklearn.model_selection.train_test_split을 통해 train데이터, validation데이터, test데이터 나누기
X_, X_test, y_, y_test=train_test_split(X,y,test_size=.2)
X_train, X_validation, y_train, y_validation=train_test_split(X_,y_)

In [10]:
#2-1-2.one-hot을 위해 tensorflow.keras.utils.to_categorical을 통해 y값들 바꾸기
y_train = to_categorical(y_train)
y_validation = to_categorical(y_validation)
y_test = to_categorical(y_test)

In [11]:
#2-2. 네트워크 설계하기(CNN 모델)
#2-2-1. VGG16모델(이미지 100가지를 분류한 모델(pretrained model)로, 이미 훈련이 되어 있어 이미지의 특징을 추출에 탁월함) (전이학습) 
conv_base=VGG16(include_top=False, input_shape=(150,150,3))  

In [14]:
#2-2-2. VGG16을 통해 얻은 이미지의 특징점 추출 및 저장
X_train_trans=conv_base.predict(X_train)
X_validation_trans=conv_base.predict(X_validation)

57/57 [==============================] - 43s 751ms/step


In [15]:
#2-2-3. 나만의 모델 설계하기
model = Sequential([
    Flatten(input_shape=X_train_trans.shape[1:]),  
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(), 
    Dense(13, activation='softmax')
])

In [16]:
#2-3. compile하기
optimizer = Adam()
loss='categorical_crossentropy'
metrics='accuracy'
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [ ]:
#2-4. fit하기(훈련하기)
model.fit(X_train_trans,y_train,batch_size=32,validation_data=(X_validation_trans,y_validation),epochs=200)

Epoch 1/200
169/169 [==============================] - 20s 102ms/step - loss: 1.9452 - accuracy: 0.3600 - val_loss: 1.3007 - val_accuracy: 0.5661
Epoch 2/200
169/169 [==============================] - 18s 104ms/step - loss: 1.2838 - accuracy: 0.5557 - val_loss: 0.8965 - val_accuracy: 0.6217
Epoch 3/200
169/169 [==============================] - 16s 97ms/step - loss: 1.0481 - accuracy: 0.6218 - val_loss: 0.8111 - val_accuracy: 0.6933
Epoch 4/200
169/169 [==============================] - 16s 94ms/step - loss: 0.9170 - accuracy: 0.6626 - val_loss: 0.7293 - val_accuracy: 0.7217
Epoch 5/200
169/169 [==============================] - 16s 94ms/step - loss: 0.8503 - accuracy: 0.6872 - val_loss: 0.6417 - val_accuracy: 0.7522
Epoch 6/200
169/169 [==============================] - 16s 98ms/step - loss: 0.7739 - accuracy: 0.7197 - val_loss: 0.6143 - val_accuracy: 0.7794
Epoch 7/200
169/169 [==============================] - 17s 102ms/step - loss: 0.7187 - accuracy: 0.7393 - val_loss: 0.5563 - val

In [ ]:
#2-5. 평가하기
model.evaluate(conv_base.predict(X_test),y_test)

In [ ]:
#3.완성된 모델 저장하기
save_model(model, 'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\model_sky.h5')
save_model(conv_base,'C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\VGG16_model_sky.h5')

#4.저장한 모델 불러오기
new_model = load_model('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\\model_sky.h5')
new_conv = load_model('C:\\Users\\user\\Desktop\\sprint_test_II\\_data_analysis\\code\\self_model\\VGG16_model_sky.h5')

In [ ]:
#5. 완성한 모델을 임의의 테스트 이미지파일에 적용해보기
#5-1. 임의의 테스트 이미지파일을 준비시키기 위한 함수
def model_predict(x, conv_base):
    x1 = cv.resize(x,(150,150))   #cv를 통해 이미지 사이즈 조정
    x1 = x1[:,:,:3]               #테스트 이미지파일의 색에 대한 변수가 RGB가 아닌 RGBA일 경우를 대비하여 slicie함.
    x1 = x1.reshape(1,150,150,3)  #모델 설계당시 input하기로 한 형태로 shape을 바꿔줌. 
    answer=np.argmax(new_model.predict(new_conv.predict(x1)))
    return answer